In [1]:
# imports and env't config 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import json
import requests 
from imp import reload 
import seaborn as sns
import Util as util; reload(util)
import warnings
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import time 

import Cdf
import Pmf
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
apr = pd.read_csv('../dataset/monroe_weblar_merged_201804.csv', sep = ',', encoding = "ISO-8859-1")
jul = pd.read_csv('../dataset/monroe_weblar_merged_201807.csv', sep = ',', encoding = "ISO-8859-1")
apr['atf'] = -1
apr['tlsHandshake'] = -1

frames = [apr, jul]
df = pd.concat(frames, ignore_index =  True)

df = df[df['operator'] != 'ice.net']
df = df[df['operator'] != 'Telenor']
df['operator'] = df['operator'].apply(lambda x: util.change_op_cc(x))
df = df[df['url'] != 'https://www.ebay.com/']
df = df[df['url'] != 'https://www.amazon.com/']
df['url'] = df['url'].apply(lambda x: util.get_base_url_name(x))

df = df[df['3sec_rendering_time'] > 6000]
df = df[df['10sec_rendering_time'] > 0]
df = df[df['14sec_rendering_time'] > 0]
df = df[df['atf_integral'] > 0]
df = df[df['atf_instant'] > 0]

df['object_size_total'] = df['object_size_total'].apply(lambda x: x/1024) #MB
df['object_size_total'] = df['object_size_total'].apply(lambda x: x/1024)

speed_df = pd.read_csv('../dataset/monroe_weblar_merged_with_speed.csv', sep = ',', encoding = "ISO-8859-1")
speed_df = speed_df[speed_df['gps_speed'] > 0]
mobile_msmt_id = set(speed_df['measurement_id'])
df_se_mobile = df[df['node_id'].isin(util.se_mobile)]
mobile_msmt_id = mobile_msmt_id.union(set(df_se_mobile['measurement_id']))

df['is_mobile'] = df['measurement_id'].apply(lambda x: util.is_from_mobile(x, mobile_msmt_id))


df_mobile = df[df['measurement_id'].isin(mobile_msmt_id)]
df_stationary = df[~df['measurement_id'].isin(mobile_msmt_id)] #isnot in

df_no_mobile = df[df['node_id'].isin(util.no_mobile)]
df_se_mobile = df[df['node_id'].isin(util.se_mobile)]
df_no_stationary = df[df['node_id'].isin(util.no_stationary)]
df_se_stationary = df[df['node_id'].isin(util.se_stationary)]

In [3]:
df[['url', '10sec_rendering_time','atf_integral', 'atf_instant']].head()

,url,10sec_rendering_time,atf_integral,atf_instant
3,google,14200.0,8280.00000,3312.4
4,google,14100.0,7663.00000,2790.3
5,yahoo,15900.0,5677.84810,12722.6
7,google,14500.0,6647.00000,3808.6
9,yahoo,7100.0,6354.91464,15276.2


In [4]:
for u in set(df['url']):
    print('{} {}'.format(u, len(df[df['url'] == u]['measurement_id'])))

youtube 8
google 1589
microsoft 575
wikipedia 1273
yahoo 318
reddit 240
bbc 580


In [5]:
df.count()

10sec_rendering_time    4583
14sec_rendering_time    4583
3sec_rendering_time     4583
as_holders              4570
ases                    4570
atf                     4583
atf_instant             4583
atf_integral            4583
bi_atf                  4583
bi_plt                  4583
cell_id                 4583
count_pixels            4583
dnsLookup               4583
dom                     4583
fetchTime               4583
ii_atf                  4583
ii_plt                  4583
img_atf                 4583
ip_ttl                  4583
locaton_area_code       4583
measurement_id          4583
node_id                 4583
number_of_css           4583
number_of_img           4583
number_of_misc          4583
number_of_script        4583
number_of_total         4583
object_size_css         4583
object_size_img         4583
object_size_misc        4583
object_size_script      4583
object_size_total       4583
oi_atf                  4583
oi_plt                  4583
operator      

In [6]:
for u in set(df['url']):
    d = df[df['url'] == u] 
    print('{}: {} | {} | {} | {} | {} | {}'.format(u, 
                                    np.median(d['object_size_total']), 
                                    np.median(d['number_of_total']),
                                   np.median(d['img_atf']),
                                    np.median(d['number_of_img']),
                                   np.median(d['number_of_script']),
                                   np.median(d['number_of_css'])))

youtube: 3.2046961784362793 | 81.0 | 15.0 | 51.0 | 9.0 | 7.0
google: 0.2060384750366211 | 120.0 | 33.0 | 87.0 | 11.0 | 11.0
microsoft: 0.9887466430664062 | 40.0 | 1.0 | 13.0 | 9.0 | 4.0
wikipedia: 1.0257654190063477 | 43.0 | 2.0 | 30.0 | 5.0 | 2.0
yahoo: 2.5796165466308594 | 150.0 | 7.0 | 28.5 | 85.0 | 4.0
reddit: 0.3106656074523926 | 122.0 | 38.0 | 46.0 | 13.0 | 10.0
bbc: 0.31167030334472656 | 102.0 | 9.0 | 50.0 | 31.0 | 4.0
